In [45]:
# Librerias necesarias
import os
import sys
import json
import random
import numpy as np
import pandas as pd
from copy import deepcopy
from generacion_pacientes import generar_pacientes
from collections import deque
# Add the parent directory to sys.path to allow relative imports
sys.path.append(os.path.abspath(os.path.join('..', '1. codigo analisis')))
import parametros as p

In [16]:
# Cargo los datos necesarios y transformo las llaves nuevamente a int (que se habian vuelto str)
incertidumbre = {}
for hospital in range(0,4): # 0 son llegadas a WL
    incertidumbre[hospital] = {}
    for requerimiento in range(1,4):
        incertidumbre[hospital][requerimiento] = {}

with open("resultados incertidumbre/incertidumbre_simulada.json", "r") as file:
    incertidumbre_keys_str = json.load(file)

for hospital in range(0,4): # 0 son llegadas a WL
    for requerimiento in range(1,4):
        for grd in range(1,9):
            incertidumbre[hospital][requerimiento][grd] = []
            lista = incertidumbre_keys_str[str(hospital)][str(requerimiento)][str(grd)]
            if lista != []:
                for data in lista:
                    arreglado = {
                    'TI': data["TI"],
                    'camino': {1: data['camino']["1"], 2: data['camino']["2"], 3: data['camino']["3"]},
                    'espera': {1: data['espera']["1"], 2: data['espera']["2"], 3: data['espera']["3"]}
                    }
                    incertidumbre[hospital][requerimiento][grd].append(arreglado)
            else:
                incertidumbre[hospital][requerimiento][grd] = []


In [17]:
class Paciente:
    CONTADOR_ID = 1  # Variable de clase para contar el número de Pacientes creados

    def __init__(self, hospital_llegada, requerimiento_inicial, grd, datos_incertidumbre):
        # Atributos del paciente al llegar, no cambian
        self.id = Paciente.CONTADOR_ID  # Asigna un ID único al Paciente
        self.hospital_llegada = hospital_llegada
        self.requerimiento_inicial = requerimiento_inicial
        self.grd = grd
        self.ti_inicial = datos_incertidumbre["TI"]
        self.camino = deepcopy(datos_incertidumbre["camino"])
        self.espera = deepcopy(datos_incertidumbre["espera"])
        Paciente.CONTADOR_ID += 1  # Incrementa el contador de Pacientes

        # Diccionarios parametros unidades y hospitales inverso (para simplificar el acceso)
        self.dict_hospitales = self.invertir_dict(p.dict_hospitales)
        self.dict_unidades = self.invertir_dict(p.dict_unidades)

        # Atributos que cambian a lo largo de la simulación
        self.log_eventos = []
        self.ti_evento_actual = datos_incertidumbre["TI"]
        self.tiempo_actual = datos_incertidumbre["TI"]
        self.tiempo_ultima_evolucion = 0
        self.hospital_actual = hospital_llegada
        self.unidad_actual = self.unidad_llegada()
        self.unidad_requerida = self.requerimiento_inicial
        self.esperando = True # True si esta esperando y no siendo tratado (revisar en cada ciclo)
        self.costo_social = 0
        self.costos_operativos = 0
        
    # Necesarias para cuando se crean los pacientes
    def invertir_dict(self, diccionario):
        invertido = {v: k for k, v in diccionario.items()}
        return invertido
    
    def unidad_llegada(self):
        if self.hospital_llegada == 0:
            unidad_actual = p.dict_unidades["WL"]
        else:
            unidad_actual = p.dict_unidades["ED"]
        return unidad_actual
    
    # Necesarias durante la simulación
    # Guarda los eventos que le ocurren al paciente en el log_eventos
    def agregar_log_evento(self): # Revisado, funciona bien

        if self.hospital_actual == 0 and self.unidad_actual == p.dict_unidades["WL"]:
            hospital = "WL"
        elif self.unidad_actual == p.dict_unidades["PS"]:
            hospital = "PS"
        else:
            hospital = self.dict_hospitales[self.hospital_actual]

        if self.tiempo_ultima_evolucion == 0:
            tiempo_final = self.tiempo_actual
        elif self.tiempo_ultima_evolucion != 0:
            tiempo_final = self.ti_evento_actual + self.tiempo_ultima_evolucion

        new_row = {
            'ID': self.id,
            'MS_GRD': self.grd,
            'UBICACIÓN': hospital + "_" + self.dict_unidades[self.unidad_actual],
            'TI': self.ti_evento_actual,
            'TF': tiempo_final,
            'HOSPITAL': hospital,
            'UNIDAD': self.dict_unidades[self.unidad_actual]
        }
        self.log_eventos.append(new_row)
    
    # Metodos para calcular los distintos costos en los que incurre el paciente (social y operativo)
    def costo_espera(self, tiempo_espera = 1, agregar = False): # Revisado, funciona bien
        if self.unidad_actual == p.dict_unidades["WL"]: 
            costo = p.dict_costo_espera_wl[self.grd][self.requerimiento_inicial] * tiempo_espera
        elif self.unidad_actual == p.dict_unidades["GA"]: 
            costo = p.dict_costo_espera_ga[self.hospital_actual][self.grd][self.requerimiento_inicial] * tiempo_espera
        elif self.unidad_actual == p.dict_unidades["ED"]: 
            costo = p.dict_costo_espera_ed[self.hospital_actual][self.grd][self.requerimiento_inicial] * tiempo_espera
        elif self.unidad_actual in (p.dict_unidades["OR"], p.dict_unidades["ICU"], p.dict_unidades["SDU/WARD"]): # (1, 2, 3)
            costo = p.dict_costo_espera_hospitalizado[self.hospital_actual][self.grd][self.unidad_actual][self.unidad_requerida] * tiempo_espera
        else:
            costo = 0
        if agregar:
            self.costo_social += costo
        return costo

    def costo_traslado(self, hospital_destino, agregar = False): # Revisado, funciona bien
        costo = p.dict_costo_traslado[self.hospital_actual][hospital_destino][self.grd][self.requerimiento_inicial]
        if agregar:
            self.costos_operativos += costo
        return costo
    
    def costo_desvio(self, agregar = False): # Revisado, funciona bien
        if self.unidad_actual == p.dict_unidades["ED"]:
            costo = p.dict_costo_derivar_ed[self.hospital_actual][self.grd][self.requerimiento_inicial]
        elif self.unidad_actual == p.dict_unidades["WL"]: 
            costo = p.dict_costo_derivar_wl[self.grd][self.requerimiento_inicial]
        if agregar:
            self.costos_operativos += costo
        return costo

    # Metodo para cambio de unidad o PS, creo el log correspondiente antes de cambiar la unidad
    def cambiar_unidad(self, hospital: int, id_unidad: int): # Revisado, funciona bien
        # Agrego el evento al log
        # Esto toma como TF el tiempo actual que es el TI de la unidad destino
        self.agregar_log_evento()

        # Antes de cambiar cualquier valor, veo si hubieron gastos operativos
        if id_unidad == p.dict_unidades["PS"]:
            self.costo_desvio(agregar = True)
        elif self.unidad_actual == p.dict_unidades["ED"] and self.hospital_actual != hospital and id_unidad == p.dict_unidades["ED"]:
            self.costo_traslado(hospital, agregar = True)

        # Actualizo todo lo necesario para el cambio de unidad
        # Actualizo el tiempo de evento actual, osea el TI en la nueva unidad
        self.ti_evento_actual = self.tiempo_actual
        # Actualizo el hospital actual
        self.hospital_actual = hospital
        # Actualizo la unidad actual
        self.unidad_actual = id_unidad
        # Caso de termino de la simulacion
        if self.unidad_actual == p.dict_unidades["PS"]:
            self.agregar_log_evento()
        # Corroborar si el cambio modifico el estado de espera o tratamiento
        self.espera_o_tratamiento()

    # Actualizar el tiempo implica si o si que el paciente estuvo un ciclo mas en la unidad actual
    # Nunca se debera actualizar el tiempo si ya esta en PS o END
    # Nunca se debera actualizar el tiempo si su unidad requerida es END
    def actualizar_tiempo(self): # Revisado, funciona bien
        self.tiempo_actual += 1
        costo_espera = 0
        if self.esperando:
            self.costo_espera(agregar = True)
            costo_espera = self.costo_espera()
        self.espera_o_tratamiento()
        print(f"Paciente {self.id} | Tiempo actual: {self.tiempo_actual} | Costo espera: {costo_espera} | Esperara?: {self.esperando}")
    
    # Ve estado actual, requerimiento y modifica esperando acordemente, tambien modifica unidad_requerida
    def espera_o_tratamiento(self): # Revisado, funciona bien
        if self.unidad_actual  in (p.dict_unidades["OR"], p.dict_unidades["ICU"], p.dict_unidades["SDU/WARD"]):
            if self.unidad_requerida == self.unidad_actual:
                tiempo_siendo_atendido = self.tiempo_actual - self.ti_evento_actual
                if tiempo_siendo_atendido <  self.espera[self.hospital_actual][0]:
                    self.esperando = False
                elif tiempo_siendo_atendido ==  self.espera[self.hospital_actual][0]:
                    self.esperando = True

                    # Elimino la unidad anteriormente requerida y su tiempo de evolucion (este lo guardo)
                    self.camino[self.hospital_actual].pop(0)
                    self.tiempo_ultima_evolucion = self.espera[self.hospital_actual].pop(0)
                    if len(self.camino[self.hospital_actual]) > 0:
                        self.unidad_requerida = self.camino[self.hospital_actual][0]
                    # Si no hay mas unidades en la lista, el paciente es dado de alta, requiere END que es salir
                    else:
                        self.unidad_requerida = p.dict_unidades["END"]
            else:
                self.esperando = True
        elif self.unidad_actual  in (p.dict_unidades["WL"], p.dict_unidades["GA"], p.dict_unidades["ED"]):
            self.esperando = True
        else: # Para PS y END, nunca deberia ser llamada esta funcion desde esas unidades
            pass

    def __str__(self): # Revisado, funciona bien
        if self.hospital_actual == 0 and self.unidad_actual == p.dict_unidades["WL"]:
            hospital = "WL"
        elif self.unidad_actual == p.dict_unidades["PS"]:
            hospital = "PS"
        else:
            hospital = self.dict_hospitales[self.hospital_actual]
        return f"Paciente {self.id} | Requerimiento: {self.dict_unidades[self.unidad_requerida]} | GRD: {self.grd} | {hospital} | Unidad: {self.dict_unidades[self.unidad_actual]} | Social: {self.costo_social} y Operativo: {self.costos_operativos} | Tiempo actual: {self.tiempo_actual}"


In [18]:
class WL:
    """Funciona con varias sublistas, una por cada combinacion de requerimiento y grd,
    al usar deques las operaciones de append y popleft son O(1), lo cual lo hace mas eficiente
    al ser un sistema de colas FIFO"""
    def __init__(self, requerimientos: list, grds: list):
        self.sub_listas = {}
        self.costo_acumulado_sub_listas = {}
        self.crear_sublistas(requerimientos, grds)
    
    def crear_sublistas(self, requerimientos: list, grds: list): # Revisado, funciona bien
        for requerimiento in requerimientos:
            self.sub_listas[requerimiento] = {}
            self.costo_acumulado_sub_listas[requerimiento] = {}
            for grd in grds:
                self.sub_listas[requerimiento][grd] = deque()
                self.costo_acumulado_sub_listas[requerimiento][grd] = 0
    
    def agregar_paciente(self, paciente: Paciente): # Revisado, funciona bien
        self.sub_listas[paciente.requerimiento_inicial][paciente.grd].append(paciente)
    
    # Siempre el primero de cada sublista es el que lleva mas esperando porque llego antes
    def sacar_paciente(self, requerimiento: int, grd: int): # Revisado, funciona bien
        if self.sub_listas[requerimiento][grd]:
            return self.sub_listas[requerimiento][grd].popleft()
        else:
            return None
        
    def actualizar_tiempo(self): # Revisado, funciona bien
        # Llama al método actualizar_tiempo de cada paciente de cada sublista
        for requerimiento in self.sub_listas:
            for grd in self.sub_listas[requerimiento]:
                if self.sub_listas[requerimiento][grd]:
                    for paciente in self.sub_listas[requerimiento][grd]:
                        paciente.actualizar_tiempo()
        self.acumular_costo_ciclo()      
    
    def acumular_costo_ciclo(self): # Revisado, funciona bien
        for requerimiento in self.sub_listas:
            for grd in self.sub_listas[requerimiento]:
                if self.sub_listas[requerimiento][grd]:
                    for paciente in self.sub_listas[requerimiento][grd]:
                        self.costo_acumulado_sub_listas[requerimiento][grd] += paciente.costo_espera()
                else:
                    self.costo_acumulado_sub_listas[requerimiento][grd] += 0

    def __str__(self):
        texto = "Sublistas de WL:"
        for requerimiento in self.sub_listas:
            texto += f"\nRequerimiento {requerimiento}:"
            for grd in self.sub_listas[requerimiento]:
                texto += f"\nGRD {grd}: {len(self.sub_listas[requerimiento][grd])} pacientes"
                if self.sub_listas[requerimiento][grd]:
                    texto += f" | ID Primer paciente: {self.sub_listas[requerimiento][grd][0].id}"
                else:
                    texto += " | Sin pacientes"
        
        texto += "\n\nCostos acumulados de WL:"
        for requerimiento in self.costo_acumulado_sub_listas:
            texto += f"\nRequerimiento {requerimiento}:"
            for grd in self.costo_acumulado_sub_listas[requerimiento]:
                texto += f"\nGRD {grd}: {self.costo_acumulado_sub_listas[requerimiento][grd]} pesos"
        
        return texto

In [19]:
class PS:
    """Funciona con varias sublistas, una por cada combinacion de requerimiento y grd,
    funciona simplemente como un sumidero de pacientes"""
    def __init__(self, requerimientos: list, grds: list):
        self.id_unidad = p.dict_unidades["PS"]
        self.sub_listas = {}
        self.costo_acumulado_sub_listas = {}
        self.crear_sublistas(requerimientos, grds)
    
    def crear_sublistas(self, requerimientos: list, grds: list): # Revisado, funciona bien
        for requerimiento in requerimientos:
            self.sub_listas[requerimiento] = {}
            self.costo_acumulado_sub_listas[requerimiento] = {}
            for grd in grds:
                self.sub_listas[requerimiento][grd] = deque()
                self.costo_acumulado_sub_listas[requerimiento][grd] = 0
    
    def agregar_paciente(self, paciente: Paciente): # Revisado, funciona bien
        self.sub_listas[paciente.requerimiento_inicial][paciente.grd].append(paciente)
        self.costo_acumulado_sub_listas[paciente.requerimiento_inicial][paciente.grd] += paciente.costo_desvio()
        paciente.cambiar_unidad(paciente.hospital_actual, self.id_unidad)

    def __str__(self):
        texto = "Sublistas de PS:"
        for requerimiento in self.sub_listas:
            texto += f"\nRequerimiento {requerimiento}:"
            for grd in self.sub_listas[requerimiento]:
                texto += f"\nGRD {grd}: {len(self.sub_listas[requerimiento][grd])} pacientes"
                if self.sub_listas[requerimiento][grd]:
                    texto += f" | {self.sub_listas[requerimiento][grd][0].id}"
                else:
                    texto += " | 0 pacientes"
        texto += "\n\nCostos acumulados de PS:"
        for requerimiento in self.costo_acumulado_sub_listas:
            texto += f"\nRequerimiento {requerimiento}:"
            for grd in self.costo_acumulado_sub_listas[requerimiento]:
                texto += f"\nGRD {grd}: {self.costo_acumulado_sub_listas[requerimiento][grd]} pesos"
                if self.costo_acumulado_sub_listas[requerimiento][grd]:
                    texto += f" | {self.sub_listas[requerimiento][grd][0].id}"
                else:
                    texto += " | 0 pacientes"
        return texto

In [20]:
class END:
    """
    Funciona con varias sublistas, una por cada combinación de requerimiento y grd,
    almacena pacientes terminados.
    """
    def __init__(self, requerimientos: list, grds: list):
        self.id_unidad = p.dict_unidades["END"]
        self.sub_listas = {}
        self.crear_sublistas(requerimientos, grds)

    def crear_sublistas(self, requerimientos: list, grds: list): # Revisado, funciona bien
        for requerimiento in requerimientos:
            self.sub_listas[requerimiento] = {}
            for grd in grds:
                self.sub_listas[requerimiento][grd] = deque()

    def agregar_paciente(self, paciente: Paciente): # Revisado, funciona bien
        """
        Agrega un paciente a la sublista correspondiente según su requerimiento_inicial y grd.
        """
        self.sub_listas[paciente.requerimiento_inicial][paciente.grd].append(paciente)
        paciente.cambiar_unidad(paciente.hospital_actual, self.id_unidad)

    def __str__(self):
        lines = ["Sublistas de Termino:"]
        for req, dict_grd in self.sub_listas.items():
            for grd, dq in dict_grd.items():
                lines.append(f"  - Requerimiento {req} GRD {grd}: {len(dq)} pacientes")
        return "\n".join(lines)

In [ ]:
class Unidad:
    """Clase para las unidades, se usan listas para almacenar pacientes debido a su pequeño tamaño, 
    en el peor de los casos el SDU/WARD tiene 165 pacientes, por lo que una lista es suficiente y
    es facil de iterar sobre ella. Todas las unidades de los hospitales heredan de la clase Unidad, 
    se comportan exactamente igual solo varia el id de la unidad y la capacidad, que se obtiene del 
    diccionario de capacidades"""
    def __init__(self, id_unidad, hospital, capacidad):
        self.id_unidad = id_unidad
        self.hospital = hospital
        self.capacidad = capacidad
        self.pacientes = []

    @property # Revisado, funciona bien
    def ocupacion(self):
        return len(self.pacientes)
    
    @property # Revisado, funciona bien
    def ocupacion_porcentual(self):
        return len(self.pacientes)/self.capacidad
    
    def agregar_paciente(self, paciente: Paciente): # Revisado, funciona bien
        # Si el paciente es None, no se agrega, osea que no estaba donde se queria sacar
        if paciente:
            if self.ocupacion < self.capacidad and paciente not in self.pacientes:
                # Si el paciente ya esta en la lista, no se agrega
                # Si la ocupacion es menor a la capacidad, se agrega el paciente
                self.pacientes.append(paciente)
                # Al agregar el paciente, se actualiza su unidad actual con el metodo de la clase
                paciente.cambiar_unidad(self.hospital, self.id_unidad)
                return True
            else:
                return False
        else:
            return False
        
    def sacar_paciente(self, paciente): # Revisado, funciona bien
        if self.ocupacion > 0 and paciente in self.pacientes:
            # Si el paciente no esta en la lista, no se saca
            # Si la ocupacion es mayor a 0, se saca el paciente
            # Cada instancia de paciente es unica, por lo que no hay problema al usar remove
            self.pacientes.remove(paciente) 
            return paciente
        else:
            return None
    
    def actualizar_tiempo(self): # Revisado, funciona bien
        # Llama al método actualizar_tiempo de cada paciente de la unidad
        for paciente in self.pacientes:
            paciente.actualizar_tiempo()
    
    def __str__(self):
        # Diccionario invertido para obtener el nombre de la unidad
        unit_names = {
            p.dict_unidades["OR"]: "OR",
            p.dict_unidades["ICU"]: "ICU",
            p.dict_unidades["SDU/WARD"]: "SDU/WARD",
            p.dict_unidades["GA"]: "GA",
            p.dict_unidades["ED"]: "ED"
        }
        unit_name = unit_names.get(self.id_unidad, f"Unidad {self.id_unidad}")
        description = f"{unit_name} (Hospital {self.hospital}): {self.ocupacion}/{self.capacidad} pacientes"
        if self.pacientes:
            description += f"\nPacientes: {len(self.pacientes)}, {self.ocupacion_porcentual:.2%} ocupación"
            if len(self.pacientes) <= 20:  # Only show details for a small number of patients
                for i, paciente in enumerate(self.pacientes):
                    description += f"\n  {i+1}. Paciente ID {paciente.id}, GRD {paciente.grd}"
        return description

class Or(Unidad):
    def __init__(self, hospital):
        id_unidad = p.dict_unidades["OR"]
        capacidad = p.dict_capacidades[hospital][id_unidad]
        super().__init__(id_unidad, hospital, capacidad)

class Icu(Unidad):
    def __init__(self, hospital):
        id_unidad = p.dict_unidades["ICU"]
        capacidad = p.dict_capacidades[hospital][id_unidad]
        super().__init__(id_unidad, hospital, capacidad)

class SduWard(Unidad):
    def __init__(self, hospital):
        id_unidad = p.dict_unidades["SDU/WARD"]
        capacidad = p.dict_capacidades[hospital][id_unidad]
        super().__init__(id_unidad, hospital, capacidad)

class Ga(Unidad):
    def __init__(self, hospital):
        id_unidad = p.dict_unidades["GA"]
        capacidad = p.dict_capacidades[hospital][id_unidad]
        super().__init__(id_unidad, hospital, capacidad)

class Ed(Unidad):
    def __init__(self, hospital):
        id_unidad = p.dict_unidades["ED"]
        capacidad = p.dict_capacidades[hospital][id_unidad]
        super().__init__(id_unidad, hospital, capacidad)

In [22]:
class Hospital:
    """Representa un hospital con una instancia de cada unidad disponible."""
    def __init__(self, hospital_id):
        self.hospital = hospital_id
        # Instanciar una unidad de cada tipo
        self.OR = Or(hospital_id)
        self.ICU = Icu(hospital_id)
        self.SDU_WARD = SduWard(hospital_id)
        self.GA = Ga(hospital_id)
        self.ED = Ed(hospital_id)
        # Diccionario para acceso y recorridos
        self.unidades = {
            p.dict_unidades["OR"]: self.OR,
            p.dict_unidades["ICU"]: self.ICU,
            p.dict_unidades["SDU/WARD"]: self.SDU_WARD,
            p.dict_unidades["GA"]: self.GA,
            p.dict_unidades["ED"]: self.ED,
        }

    def actualizar_tiempo(self): # Revisado, funciona bien
        # Llama al método actualizar_tiempo de cada unidad del hospital
        for unidad in self.unidades.values():
            unidad.actualizar_tiempo()
    
    def ocupacion(self): # Revisado, funciona bien
        # Devuelve la ocupación de todo el hospital
        ocupacion = {
            unidad_id: (unidad.ocupacion_porcentual, unidad.ocupacion) 
            for unidad_id, unidad in self.unidades.items()
        }
        return ocupacion
    
    def agregar_paciente(self, paciente: Paciente, unidad: int): # Revisado, funciona bien
        # Agrega un paciente a la unidad correspondiente.
        if unidad in self.unidades:
            return self.unidades[unidad].agregar_paciente(paciente)
        else:
            raise ValueError(f"Unidad {unidad} no válida en el hospital {self.hospital}.")
    
    def sacar_paciente(self, paciente: Paciente, unidad: int): # Revisado, funciona bien
        # Saca un paciente de la unidad correspondiente.
        if unidad in self.unidades:
            return self.unidades[unidad].sacar_paciente(paciente)
        else:
            raise ValueError(f"Unidad {unidad} no válida en el hospital {self.hospital}.")
    
    def __str__(self):
        lines = [f"Hospital {self.hospital}:"]
        for unidad_id, unidad in self.unidades.items():
            lines.append(f"  - {unidad_id}: {unidad.ocupacion}/{unidad.capacidad}")
        return "\n".join(lines)

In [ ]:
class Modelo:

    def __init__(self):
        pass

    def tomar_decisiones(self, estado_simulacion):
        return "decisiones"
    
    def __str__(self):
        pass

In [ ]:
class Simulacion:

    def __init__(self, T_max, seed, ciclos):
        """Se empieza generando los pacientes con la semilla de random
        y la cantidad de ciclos que se desean crear pacientes"""
        generar_pacientes(seed, ciclos)
        self.T_max = T_max
        self.pacientes_separados_por_llegada = self.cargar_pacientes_separados_por_llegada()
        print("Pacientes separados por llegada cargados")
        self.ciclos = ciclos
        self.modelo = Modelo()
        self.hospital_1 = Hospital(1)
        self.hospital_2 = Hospital(2)
        self.hospital_3 = Hospital(3)
        self.wl = WL([1, 2, 3], [5, 6, 7, 8])
        self.hospitales = { # Para acceder a los hospitales por su id
            0: self.wl,
            p.dict_hospitales["Hospital_1"]: self.hospital_1,
            p.dict_hospitales["Hospital_2"]: self.hospital_2,
            p.dict_hospitales["Hospital_3"]: self.hospital_3
        }
        self.ps = PS([1, 2, 3], [1, 2, 3, 4, 5, 6, 7, 8])
        self.end = END([1, 2, 3], [1, 2, 3, 4, 5, 6, 7, 8])
        self.unidades_termino = { # Para acceder a las unidades de termino por su id
            p.dict_unidades["PS"]: self.ps,
            p.dict_unidades["END"]: self.end
        }
        self.budget = p.budget
        self.tasa_descuento = p.tasa_descuento
        print("Clase Simulacion instanciada")

    # Funcion necesaria al momento de instanciar la clase
    def cargar_pacientes_separados_por_llegada(self): # Revisado, funciona bien
        # Cargo los datos necesarios y transformo las llaves nuevamente a int (que se habian vuelto str)
        incertidumbre = {}
        for hospital in range(0,4): # 0 son llegadas a WL
            incertidumbre[hospital] = {}
            for requerimiento in range(1,4):
                incertidumbre[hospital][requerimiento] = {}

        with open("resultados incertidumbre/incertidumbre_simulada.json", "r") as file:
            incertidumbre_keys_str = json.load(file)

        for hospital in range(0,4): # 0 son llegadas a WL
            for requerimiento in range(1,4):
                for grd in range(1,9):
                    incertidumbre[hospital][requerimiento][grd] = []
                    lista = incertidumbre_keys_str[str(hospital)][str(requerimiento)][str(grd)]
                    if lista != []:
                        for data in lista:
                            arreglado = {
                            'TI': data["TI"],
                            'camino': {1: data['camino']["1"], 2: data['camino']["2"], 3: data['camino']["3"]},
                            'espera': {1: data['espera']["1"], 2: data['espera']["2"], 3: data['espera']["3"]}
                            }
                            incertidumbre[hospital][requerimiento][grd].append(arreglado)
                    else:
                        incertidumbre[hospital][requerimiento][grd] = []

        # Se instancian a todos los pacientes a partir de los datos de incertidumbre
        pacientes = {}
        lista_pacientes = []
        # 0 son llegadas a WL
        for hospital in range(0,4): 
            pacientes[hospital] = {}
            for requerimiento in range(1,4):
                pacientes[hospital][requerimiento] = {}
                for grd in range(1,9):
                    pacientes[hospital][requerimiento][grd] = []
                    cantidad_pacientes = len(incertidumbre[hospital][requerimiento][grd])
                    if cantidad_pacientes != 0:
                        for i in range(cantidad_pacientes):
                            paciente = Paciente(hospital, requerimiento, grd, incertidumbre[hospital][requerimiento][grd][i]) # i es el index de la lista
                            pacientes[hospital][requerimiento][grd].append(paciente)
                            lista_pacientes.append(paciente)
                    else:
                        pacientes[hospital][requerimiento][grd] = []

        # Se generan tantas listas como dias con llegadas haya
        pacientes_separados_por_llegada = {}
        for paciente in lista_pacientes:
            dia = paciente.ti_inicial
            if dia not in pacientes_separados_por_llegada:
                pacientes_separados_por_llegada[dia] = []
            pacientes_separados_por_llegada[dia].append(paciente)

        return pacientes_separados_por_llegada
    
    # Funciones necesarias al momento de simular
    def agregar_pacientes_ciclo_a_wl(self, ciclo): # Revisado, funciona bien
        pacientes_ciclo = self.pacientes_separados_por_llegada.get(ciclo, [])
        for paciente in pacientes_ciclo:
            # hospital 0 es WL
            if paciente.hospital_llegada == 0:
                self.wl.agregar_paciente(paciente)
            else:
                pass

    def agregar_pacientes_ciclo_a_ed(self, ciclo): # Revisar bien tema de la capacidad de las ED, puede arrojar errores
        pacientes_ciclo = self.pacientes_separados_por_llegada.get(ciclo, [])
        for paciente in pacientes_ciclo:
            # si hospital es 1, 2 o 3 llegan a ED
            if paciente.hospital_llegada != 0:
                self.hospitales[paciente.hospital_llegada].agregar_paciente(paciente, paciente.unidad_actual)
            else:
                pass

    def sacar_paciente(self, paciente): # Revisar
        # Retorna el paciente que se saca de la unidad, o None si no se pudo sacar
        if paciente.hospital_actual == 0:
            return self.hospitales[paciente.hospital_actual].sacar_paciente(paciente.unidad_requerida, paciente.grd)  
        else:
            return self.hospitales[paciente.hospital_actual].sacar_paciente(paciente, paciente.unidad_actual)

    def agregar_paciente(self, paciente, hospital, unidad): # Revisar
        # Retorna True si se agrega el paciente, False si no
        if unidad not in (p.dict_unidades["PS"], p.dict_unidades["END"]):
            return self.hospitales[hospital].agregar_paciente(paciente, unidad)
        else:
            # No retornan nada porque siempre se agrega el paciente
            self.unidades_termino[unidad].agregar_paciente(paciente)
            return True

    def implementar_decisiones(self, decisiones: list): # Revisar
        """Las decisiones son una lista de diccionarios, cada uno con la siguiente estructura:
        {
            paciente: {"hospital": hospital, "unidad": unidad},
            ...
            paciente: {"hospital": hospital, "unidad": unidad},
        }
        Estas se deben implementar en el orden de la lista, se sacan a todos los pacientes de su
        unidad actual y se cambian a la unidad y hospital que se indica en el diccionario. Luego se 
        pasa al siguiente diccionario y se repite el proceso. Se hace de esta manera porque a veces
        puede ocurrir que dos pacientes tengan que ser cambiados entre si, por lo que los cambios
        deben ser simultaneos por cada diccionario, para sacar a un paciente no necesito saber donde
        esta ya que cada paciente contiene su unidad y hospital actual.
        """
        for decision in decisiones:
            # Saco a todos los pacientes de su unidad actual
            for paciente in decision:
                sacado = self.sacar_paciente(paciente)
                # Esto solo para pruebas, despues se elimina
                if sacado is paciente:
                    print(f"Paciente {paciente.id} sacado exitosamente")
                else:
                    print(f"Paciente {paciente.id} no se pudo sacar")
            # Agrego a todos los pacientes a su nueva unidad
            for paciente, destino in decision.items():
                if self.agregar_paciente(paciente, destino["hospital"], destino["unidad"]):
                    print(f"Paciente {paciente.id} agregado exitosamente a {destino['hospital']} en {destino['unidad']}")

    def actualizar_tiempo(self): # Revisado, funciona bien
        self.hospital_1.actualizar_tiempo()
        self.hospital_2.actualizar_tiempo()
        self.hospital_3.actualizar_tiempo()
        self.wl.actualizar_tiempo()

    def entregar_log_pacientes_terminados_como_data_frame(self): # Revisar
        log_completo = []
        for requerimiento in [1, 2, 3]:
            for grd in [1, 2, 3, 4, 5, 6, 7, 8]:
                for paciente in self.end.sub_listas[requerimiento][grd]:
                    for evento in paciente.log_eventos:
                        log_completo.append(evento)
                for paciente in self.ps.sub_listas[requerimiento][grd]:
                    for evento in paciente.log_eventos:
                        log_completo.append(evento)
        # Convertir la lista de eventos a un DataFrame
        df_log = pd.DataFrame(log_completo)
        # Ordenar el DataFrame por ID y TI y TF
        df_log.sort_values(by=['ID', 'TI', 'TF'], inplace=True)
        # Resetear el índice
        df_log.reset_index(drop=True, inplace=True)
        return df_log


    def simular(self):
        self.T = 1  # Inicializa el tiempo del sistema en 1

        # Bucle principal de simulación
        while self.T <= self.T_max:

            # Agregar pacientes a WL según el ciclo actual
            self.agregar_pacientes_ciclo_a_wl(self.T)

            # Agregar pacientes a las ED según el ciclo actual
            self.agregar_pacientes_ciclo_a_ed(self.T)

            # Entrego el estado de la simulación al modelo para tomar decisiones
            decisiones = self.modelo.tomar_decisiones(self) # La clase Simulacion se pasa como argumento

            # Se implementan las decisiones del modelo
            self.implementar_decisiones(decisiones)

            # Actualizar el tiempo de cada unidad y paciente
            self.actualizar_tiempo()
            
            # Incrementar el tiempo del sistema
            self.T += 1
        
        # Al finalizar la simulación, se entregan los logs de los pacientes terminados
        return self.entregar_log_pacientes_terminados_como_data_frame()
        
    def __str__(self):
        pass

In [37]:
simu = Simulacion(5001, 42, 5000)

La incertidumbre de los pacientes ha sido generada y guardada en el archivo JSON.
Pacientes separados por llegada cargados
Clase Simulacion instanciada


In [44]:
simu.wl = WL([1, 2, 3], [5, 6, 7, 8])
simu.agregar_pacientes_ciclo_a_wl(1)
for requerimiento in simu.wl.sub_listas:
    for grd in simu.wl.sub_listas[requerimiento]:
        for paciente in simu.wl.sub_listas[requerimiento][grd]:
            print(paciente)
            paciente.actualizar_tiempo()
            print(paciente)
            print("")

Paciente 332193 | Requerimiento: OR | GRD: 5 | WL | Unidad: WL | Social: 0 y Operativo: 0 | Tiempo actual: 1
Paciente 332193 | Tiempo actual: 2 | Costo espera: 4.4378762 | Esperara?: True
Paciente 332193 | Requerimiento: OR | GRD: 5 | WL | Unidad: WL | Social: 4.4378762 y Operativo: 0 | Tiempo actual: 2

Paciente 332194 | Requerimiento: OR | GRD: 5 | WL | Unidad: WL | Social: 0 y Operativo: 0 | Tiempo actual: 1
Paciente 332194 | Tiempo actual: 2 | Costo espera: 4.4378762 | Esperara?: True
Paciente 332194 | Requerimiento: OR | GRD: 5 | WL | Unidad: WL | Social: 4.4378762 y Operativo: 0 | Tiempo actual: 2

Paciente 347528 | Requerimiento: OR | GRD: 7 | WL | Unidad: WL | Social: 0 y Operativo: 0 | Tiempo actual: 1
Paciente 347528 | Tiempo actual: 2 | Costo espera: 5.337300461 | Esperara?: True
Paciente 347528 | Requerimiento: OR | GRD: 7 | WL | Unidad: WL | Social: 5.337300461 y Operativo: 0 | Tiempo actual: 2

Paciente 347529 | Requerimiento: OR | GRD: 7 | WL | Unidad: WL | Social: 0 y O